In [1]:
#Imports
import requests
import pandas as pd
import pandasql as psql

In [2]:
#Url da Fake Store API
url = 'https://fakestoreapi.com/'

#### Products

In [3]:
#Recuperando os dados da dimensão "products"
response = requests.get(f'{url}products')

## Criando o DataFrame products
data_products = response.json()
df_products = pd.DataFrame(data_products)

In [4]:
#Tratando a coluna aninhada 'rating'
df_rating_products = pd.json_normalize(df_products['rating'])
df_products = df_products.drop(columns=['rating']).join(df_rating_products)

In [5]:
#Separando somente as colunas que serão necessárias para análise
colunas = ['id','category']
df_products = df_products[colunas]

#### Users

In [6]:
#Recuperando os dados da dimensão "users"
response = requests.get(f'{url}users')

## Criando o DataFrame users
data_users = response.json()
df_users = pd.DataFrame(data_users)

In [7]:
# Tratando as colunas aninhadas 'name' e 'address'
df_users = df_users.drop(columns=['name', 'address']) \
                   .join(pd.json_normalize(df_users['name']) \
                   .join(pd.json_normalize(df_users['address'])))

In [8]:
#Separando as colunas que serão necessárias para análise
cols = ['id','username','firstname','lastname']
df_users = df_users[cols]

#### Cart

In [9]:
#Recuperando os dados da fato Cart
response = requests.get(f'{url}carts')

## Criando o DataFrame Cart
data_cart = response.json()
df_cart = pd.DataFrame(data_cart)

In [10]:
df_cart_exploded = df_cart.explode('products')

In [11]:
#df_cart_exploded
df_cart_exploded_normalized = pd.json_normalize(df_cart_exploded['products'])
df_cart = df_cart_exploded.drop(columns=['products']).join(df_cart_exploded_normalized)

In [12]:
#Criando o DataFrame final
query = """
    SELECT
        u.id as user_id,
        u.firstname || ' ' || u.lastname as user_name,
        MAX(c.date) as date_last_addition,
        p.category,
        SUM(c.quantity)
    FROM df_users u
        LEFT JOIN df_cart c ON u.id = c.userId
        LEFT JOIN df_products p ON c.productId = p.id
    GROUP BY 
        u.id
"""
df_analise = psql.sqldf(query, locals())

In [13]:
df_analise

,user_id,user_name,date_last_addition,category,SUM(c.quantity)
0,1,john doe,2020-03-02T00:00:00.000Z,men's clothing,15.0
1,2,david morrison,2020-03-01T00:00:00.000Z,men's clothing,12.0
2,3,kevin ryan,2020-03-01T00:00:00.000Z,men's clothing,24.0
3,4,don romer,2020-03-01T00:00:00.000Z,jewelery,4.0
4,5,derek powell,None,None,NaN
5,6,david russell,None,None,NaN
6,7,miriam snyder,None,None,NaN
7,8,william hopkins,2020-03-01T00:00:00.000Z,men's clothing,2.0
8,9,kate hale,None,None,NaN
9,10,jimmie klein,None,None,NaN


In [14]:
!pip list pandas

Package                       Version
----------------------------- ------------
alembic                       1.12.0
altair                        5.1.2
anyio                         4.0.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.0
async-generator               1.10
async-lru                     2.0.4
attrs                         23.1.0
Babel                         2.13.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
beautifulsoup4                4.12.2
bleach                        6.1.0
blinker                       1.6.3
bokeh                         3.3.0
boltons                       23.0.0
Bottleneck                    1.3.7
Brotli                        1.1.0
cached-property               1.5.2
certifi                       2023.7.22
certipy                       0.1.3
cffi                          1.16.0
charset-normalizer            3.3.0
click   